# UTSA CS 4973/5483: Assignment-3

Spring 2021


**Last Name - First Name - (abc123)**






## Learning Objectives

*   Epipolar Geometry
*   Triangulation
*   Sparse 3D Reconstruction

# Overview

Consider that you are given two stereo images. The end goal is to perform **Sparse 3D Reconstruction using Epipolar Geometry**. Following is the overview of the sparse 3D reconstruction process:


1.   Given two stereo images, find sparse feature point correspondences ( > 8 ).
2.   Use these correspondences to estimate the Fundamental Matrix (F) using the 8-Point Algorithm.
3.   Intrinsic matrices establish the relationship between the image and camera coordinates. These matrices are fixed for a given camera. Calibration is performed once using traditional checkerboard pattern techniques to obtain Intrinsic Matrix (K).
4.   Calculate the Essential Matrix (E) using the the fundamental matrix (F) and the camera intrinsic matrices (K1 & K2).
5.   To align the two camera views, the extrinsic matrix of the first camera (extr1) is considered to be Identity. The extrinsic matrix of the second camera (extr2) establishes the transformation between the two camera cameras.
6.   extr1 -> rotation (R1) = Identity, translation (T1) = zero. Use extr1 and K1 to obtain the projection matrix of the first camera (P1).
7.   extr2 -> Use the Essential matrix (E) to predict the extrinsic matrix of the second camera (extr2). There will be 4 estimates of which the one for which most of the projected 3D points are in front of both cameras. Use this extrinsic matrix to obtain the projection matrix of the second camera (P2).
8.   The camera projection matrices (P1 & P2) can be decomposed into the rotation matrices (R1 & R2) and the camera translation vectors (T1 & T2).
9.   Use the projection matrices P1 & P2 and given points to perform triangulation and obtain the sparse 3D reconstruction of the image.
10.   Validate the sparse 3D reconstruction by computing the reprojection error.


In order to obtain a depth value for the entire image (almost all pixels) we need to perform **Dense 3D Reconstruction using Stereo Rectification -- out of scope of this assignment**. Following is the overview of the dense 3D reconstruction process:

1.   The steps 1-8 mentioned above are followed to obtain these values -- Fundamental matric (F), Essential matrix (E), projection matrices (P1 & P2), rotation matrices (R1 & R2), translation vectors (T1 & T2), intrinsic matrices (K1 & K2).
2.   Perform Stereo Rectification to align both the images in a way that the epipolar lines in both the images are horizontal lines.
3.   Once the images are rectified, we select every pixel in the image-1 and perform a linear search for the corresponding pixel in image-2 on this horizontal epipolar line.
4.   We obtain correspondences for each and every pixel which can be used to compute the disparity map between the two images.
5.   Use the disparity map to compute the depth map, which is the dense 3D reconstruction of the two stereo images.


# Sparse 3D Reconstruction

In [ ]:
# Add your imports here


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Base path where you should have all input data/images. Use this while reading.
# This is the path that will be used by the TA - if you don't have this properly, the TA will not accept the submission.
basePath = "/content/drive/My Drive/Colab Notebooks/Computer Vision/images/"

## Sparse Feature Point Correspondence
(20 points)

In [ ]:
# Read the two images I1 & I2 and convert them to grayscale
# Plot the two images next to each other as subplots with the size of the entire figure as 20x10


In [ ]:
# Perform ORB feature detection
# Plot the features on the two images next to each other as subplots with the size of the entire figure as 20x10


In [ ]:
# Perform feature matching and extract only the top 72 matches
# Obtain the correspondences in terms of pixel locations in the left and right image. Name these correspondences as 'pts1' & 'pts2'
# Plot the matches on the two images next to each other as subplots with the size of the entire figure as 20x10


In [ ]:
# Load manually cleaned point correspondences for the temple stereo images
# CODE PROVIDED & COMMENTED
# pts1 & pts2 arrays to be used by the TA during grading

# test = np.load(basePath+"temple_points.npz")
# pts1 = test['pts1']
# pts2 = test['pts2']

## Fundamental Matrix (F) using the 8-Point Algorithm
(40 points)

In [ ]:
# Write a function for the 8-Point Algorithm.
# Input -- pts1, pts2, M (scale factor to normalize - max of the image width or height)
# Output -- Fundamental Matrix F

def eightpoint(pts1, pts2, M):
    # REPLACE with your implementation


In [ ]:
# Call the above created function for the 8-Point Algorithm
# Store and print the returned Fundamental Matrix F


## Intrinsic Matrices K-1 & K-2

In [ ]:
# Load the intrinsic matrices -- CODE PROVIDED
intrinsics = np.load(basePath+"intrinsics.npz")
K1, K2 = intrinsics['K1'], intrinsics['K2']
print(K1)
print(K2)

## Estimate Essential Matrix (E)
(5 points)

In [ ]:
# Write a function to estimate the essential matrix
# Input -- F, K1, K2
# Output -- E

def essentialMatrix(F, K1, K2):
    # REPLACE with your implementation
    

In [ ]:
# Call the above created function
# Store and print the returned Essential Matrix E


## Camera-1 Projection
(5 points)

In [ ]:
# extr1 = [R | T] with R = I and T = 0-> 3x4
# Compute and print the projection matrix of the first camera (P1) using K1 and ext1 -> 3x4


## Triangulation
(40 points)

In [ ]:
# Write a function to perform triangulation
# Input: P1 - camera-1 projection matrix -> 3x4
#        P2 - camera-2 projection matrix -> 3x4
#        pts1 - points from image-1 -> Nx2
#        pts2 - corresponding points from image-2 -> Nx2
# Output: pts3D - triangulated 3D points -> Nx3

def triangulate(P1, P2, pts1, pts2):
    # REPLACE with your implementation

    # Compute the 3D point for each point correspondence by doing the following:
    #     Create the array A consisting of 2 rows each from corresponding points from pts1 and pts2
    #     Perform SVD of 'A' to obtain the 3D point 'X'
    #     Normailze the points to a scale and append to the list of triangulated 3D points
    # Convert homogeneous 3D points to be heterogeneous 3D points


## Camera-2 Projection
(20 points)

In [ ]:
# Function to obtain the 4 possibilities for the camera-2 extrinsic matrix (extr2) -- CODE PROVIDED
# Input -- Essential Matrix E
# Ouput -- extr2

def camera2(E):
    U,S,V = np.linalg.svd(E)
    m = S[:2].mean()
    E = U.dot(np.array([[m,0,0], [0,m,0], [0,0,0]])).dot(V)
    U,S,V = np.linalg.svd(E)
    W = np.array([[0,-1,0], [1,0,0], [0,0,1]])

    if np.linalg.det(U.dot(W).dot(V))<0:
        W = -W

    extr2s = np.zeros([3,4,4])
    extr2s[:,:,0] = np.concatenate([U.dot(W).dot(V), U[:,2].reshape([-1, 1])/abs(U[:,2]).max()], axis=1)
    extr2s[:,:,1] = np.concatenate([U.dot(W).dot(V), -U[:,2].reshape([-1, 1])/abs(U[:,2]).max()], axis=1)
    extr2s[:,:,2] = np.concatenate([U.dot(W.T).dot(V), U[:,2].reshape([-1, 1])/abs(U[:,2]).max()], axis=1)
    extr2s[:,:,3] = np.concatenate([U.dot(W.T).dot(V), -U[:,2].reshape([-1, 1])/abs(U[:,2]).max()], axis=1)

    return extr2s

In [ ]:
# Call the above function to obtain the 4 possibilities for extr2 -- CODE PROVIDED
extr2s = camera2(E)
print(extr2s)

In [ ]:
# Use the 'triangulate' function to figure out the correct value for extr2 from the 4 possibilities obtained above
# For each possibility:
#     Compute P2 using K2 and extr2
#     Use this P2 to triangulate and obtain the 3D points
# The extr2 for which all the triangulated 3D points are in front of both cameras (i.e., z > 0) is the correct extr2


# REPLACE with your implementation



In [ ]:
# Print the final correct extr2 and the corresponding P2 -- CODE PROVIDED
print(extr2)
print(P2)

## Projection Matrix Decomposition
(5 points)

In [ ]:
# Call the cv2.decomposeProjectionMatrix() function to decompose the projections matrices (P1 & P2) to obtain K, R, T separately for each of them
# Print all 6 - K1 (3x3), K2 (3x3), R1 (3x3), R2 (3x3), T1 (3x1), T2 (3x1)


## 3D Reconstruction
(10 points)

In [ ]:
# Perform Sparse 3D Reconstruction
# Call the 'triangulate' function using the final camera projection matrices (P1 & P2) and the points (pts1 & pts2) to obtain the final set of 3D points (pts3D)
# Use plotly.express to plot an interactable 3D Scatter Plot for the 3D points (pts3D)


## Reprojection Error
(20 points)

In [ ]:
# Write a function to perform triangulation and obtain the reprojection error
# Input: P1 - camera-1 projection matrix -> 3x4
#        P2 - camera-2 projection matrix -> 3x4
#        pts1 - points from image-1 -> Nx2
#        pts2 - corresponding points from image-2 -> Nx2
# Output: reprojection_error -> average Eucildean error of reprojection

def reprojection_error(P1, P2, pts1, pts2):
    # REPLACE with your implementation

    # Compute the 3D point for each point correspondence by doing the following:
    #     Create the array A consisting of 2 rows each from corresponding points from pts1 and pts2
    #     Perform SVD of 'A' to obtain the 3D point 'X'
    #     Normailze the points to a scale and append to the list of triangulated 3D points

    # Reproject the 3D points using P1 to obtain pts1_reprojected
    # Reproject the 3D points using P2 to obtain pts2_reprojected
    # Normalize both and convert them to heterogeneous coordinates
    # Compare (pts1 -- pts1_reprojected) and (pts1 -- pts1_reprojected)
    # Use Eucildean distance between each point to compute and return the average reprojection error


In [ ]:
# Call the above functin to obtain and print the reprojection error for the ORB features points 'pts1' and 'pts2' -- CODE PROVIDED
print(reprojection_error(P1, P2, pts1, pts2))

#Submission Instructions



1.   Complete all tasks above
2.   Export this notebook as .ipynb
      (File > Download as ipynb)
3.   Upload the .ipynb file on Blackboard - **file MUST contain the output for ALL cells**


##Rubric

*   (20 points) Sparse feature point correspondence
*   (40 points) Fundamental Matrix (F) using the 8-Point Algorithm
*   (05 points) Estimate Essential Matrix (E)
*   (05 points) Camera-1 Projection
*   (40 points) Triangulation
*   (20 points) Camera-2 Projection
*   (05 points) Projection Matrix Decomposition
*   (10 points) 3D Reconstruction
*   (20 points) Reprojection Error